# Notebook Illustrating Initial Database ETL for Webapp
- Database Creation and Initial ETL Script: ```db.py```
- ETL Script: ```etl.py```
- SOQL Queries: ```soql_queries.py```
- Misc. ETL Functions: ```etl_functions.py```
- Database Models: ```models.py``` (note: functions have been refactored for ETL)
- NYC Open Data with Socrata (SODA API) for Drink Outside NYC Webapp
- Google Cloud Maps Geocoding API 

## SODA API
- Get API Token utilizing ```configparser```
    - Config file as "hidden" ```.ini``` file
- Create connection to Socrata with ```sodapy```
    - Parameters: domain, API Token
- Make SoQL query with ```sodapy```
    - Dataset identifier: ```pitm-atqc```
    - Only retrieve establishments with SLA License and sidewalk/openstreets/roadway seating

In [1]:
from sodapy import Socrata
from configparser import ConfigParser

In [2]:
# API Configuration
config = ConfigParser()
config.read('./.ini')
TOKEN = config.get('Socrata', 'DRINK_NYC_TOKEN')

# Connect to Socrata API
client = Socrata('data.cityofnewyork.us', TOKEN)

# Make query for establishments serving alcohol
results = client.get('pitm-atqc', where='qualify_alcohol="yes"', limit=50000)

print('Number of establishments:', len(results))

Number of establishments: 8411


In [3]:
# Sample
print(results[10]['doing_business_as_dba'])

KARIZMA LOUNGE CORP


## Google Cloud Map Platform (Geocoding)
- Retrieve and merge missing latitude and longitude data onto ```results```

In [4]:
# Create list of establishments without latitude/longitude
missing_loc = [ r for r in results if 'latitude' not in r.keys() ]

print('Number of establishments without geolocation:', len(missing_loc))

Number of establishments without geolocation: 877


In [5]:
import requests

In [6]:
# Set maps API key
MAPS_API_KEY = config.get('GCP', 'API_KEY')

In [7]:
def _get_lat_lng(address):
    '''
    Function to retrieve geolocation via Google Cloud Map Platform
    '''
    
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
    address = '+'.join(address.split())
    url = base_url + address + '&key=' + MAPS_API_KEY

    r = requests.get(url)
    
    if r.status_code == 200:
        result = r.json()
        location = result['results'][0]['geometry']['location']
        return location
    
    else:
        return {'lat': None, 'lng': None}
    
def _merge_data(result):
    '''
    Function to retrieve and merge data onto establishment data 
    '''
    
    address = result['business_address']
    location = _get_lat_lng(address)
    
    return location

In [8]:
# Sample
sample = missing_loc[10]
print('***Pre-data merge***:', sample)
print('-' * 30)

# Sample data retrieval and update
address = sample['business_address']
location = _merge_data(sample)
sample['latitude'] = location['lat']
sample['longitude'] = location['lng']
print('***Post-data merge***:', sample)

***Pre-data merge***: {'objectid': '3681', 'globalid': '{d13ac929-c53d-4298-8d8e-282e91866dbf}', 'seating_interest_sidewalk': 'roadway', 'restaurant_name': 'Tufino Pizzeria Napoletana', 'legal_business_name': 'Tufino Pizzeria Napoletana inc.', 'doing_business_as_dba': 'Tufino Pizzeria Napoletana inc.', 'bulding_number': '3608 3610', 'street': 'Ditmars Blvd', 'borough': 'Queens', 'zip': '11105', 'business_address': '3608 3610 Ditmars Blvd , Queens, NY', 'food_service_establishment': '41696092', 'roadway_dimensions_length': '20', 'roadway_dimensions_width': '8', 'roadway_dimensions_area': '160', 'approved_for_sidewalk_seating': 'no', 'approved_for_roadway_seating': 'yes', 'qualify_alcohol': 'yes', 'sla_serial_number': '1302285', 'sla_license_type': 'OP', 'landmark_district_or_building': 'no', 'healthcompliance_terms': 'yes', 'time_of_submission': '2020-06-22T14:26:00.000'}
------------------------------
***Post-data merge***: {'objectid': '3681', 'globalid': '{d13ac929-c53d-4298-8d8e-282

## Data Cleaning / Transformation
- Create capacity estimates: 6'x6' per 4 person table with approx. 6' space between tables
- Convert application submission string to datetime object
- Remove irrelevant data

In [9]:
import math
from datetime import datetime

In [10]:
def _est_capacity(result):
    '''
    Function to create capacity estimates
    '''
    
    sidewalk = result['sidewalk']
    roadway = result['roadway']
    capacity = 0
    
    if sidewalk['status']:
        l = round(sidewalk['length'] / 6)
        if l % 2 != 0:
            l = math.floor(l / 2) + 1
        else:
            l = math.floor(l / 2)
            
        w = round(sidewalk['width'] / 6)
        if w % 2 != 0:
            w = math.floor(w / 2) + 1
        else: 
            w = math.floor(w / 2)
            
        capacity += l * w * 4
    
    if roadway['status']:
        l = round(roadway['length'] / 6)
        if l % 2 != 0:
            l = math.floor(l / 2) + 1
        else:
            l = math.floor(l / 2)
            
        w = round(roadway['width'] / 6)
        if w % 2 != 0:
            w = math.floor(w / 2) + 1
        else: 
            w = math.floor(w / 2)
            
        capacity += l * w * 4
        
        return capacity

In [11]:
def clean_for_transformation(result):
    '''
    Function to clean establishment data into objects ready for further transformation
    '''
    
    if 'latitude' not in result.keys():
        _merge_data(result)
        
    establishment = {
        '_id': result['globalid'].strip('{}'),
        'name': result['doing_business_as_dba'],
        'address': result['business_address'],
        'zip': result['zip'],
        'license': result['sla_license_type'],
        'tos': datetime.strptime(result['time_of_submission'], '%Y-%m-%dT%H:%M:%S.%f'),
        'lat': float(result['latitude']),
        'lng': float(result['longitude']),
        'sidewalk': {'status': False},
        'roadway': {'status': False},
        'openstreets': {'status': False}
    }
    
    if result['approved_for_sidewalk_seating'] == 'yes':
        establishment['sidewalk']['status'] = True
        establishment['sidewalk']['length'] = int(result['sidewalk_dimensions_length'])
        establishment['sidewalk']['width'] = int(result['sidewalk_dimensions_width'])
        
    if result['approved_for_roadway_seating'] == 'yes':
        establishment['roadway']['status'] = True
        establishment['roadway']['length'] = int(result['roadway_dimensions_length'])
        establishment['roadway']['width'] = int(result['roadway_dimensions_width'])
        
    if result['seating_interest_sidewalk'] == 'openstreets':
        establishment['openstreets']['status'] = True
    
    establishment['est_capacity'] = _est_capacity(establishment)
    
    return establishment

In [12]:
# Sample
sample = clean_for_transformation(results[0])
sample

{'_id': 'F4FFE759-9440-444D-8301-D9674B698218',
 'name': 'HERE ARTS CENTER',
 'address': '145 AVENUE OF THE AMERICAS, Manhattan, NY',
 'zip': '10013',
 'license': 'RW',
 'tos': datetime.datetime(2021, 4, 22, 14, 47),
 'lat': 40.724971,
 'lng': -74.004477,
 'sidewalk': {'status': False},
 'roadway': {'status': True, 'length': 40, 'width': 6},
 'openstreets': {'status': False},
 'est_capacity': 16}

## Build Database
- Establish connection to Google Firestore database (note: Firestore implicitly creates collections)
- Utilize ```Establishment``` model for data loading

In [13]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from models import Establishment

# Use a service account
cred = credentials.Certificate('./drinkoutsidenyc-f825926aaa1f.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [14]:
# Standard single load
sample = Establishment(results[0])
result = db.collection('establishments').document(sample._id).set(dict(sample))

print(result)

update_time {
  seconds: 1621636604
  nanos: 455183000
}



In [15]:
# Batch load (initial database population)
batch = db.batch()
for r in results[:10]:
    sample = Establishment(r)
    ref = db.collection('establishments').document(sample._id)
    batch.set(ref, dict(sample))

result = batch.commit()
len(result)

10